# Serving User Recommendations with Sorted Sets

Personalization is an  important part of growing a successful  website. Personalization can
increase user traffic on  your website. Various algorithms exist to  predict what content a
user might be  interested in given their  browsing history. Many third  party systems exist
to compute  recommendations, but recommendations aren't  useful if they can't  be served to
users quickly.

In this chapter, we are going to look at  how the Redis sorted set set datatype can be used
to serve up blog recommendations. Among the topics we will cover in this chapter:

* Sorted sets versus sets
* Specifying data storage conventions 
* Storing recommendations
* Viewing Recommendations  

## Sorted Set 

Redis provides two types of set data structures, the set and the sorted set. In the
previous chapter we looked at using the Set type to calculate unique views, and in this
chapter we are going to look at using the other set type, Sorted Sets, to serve up user
recommendations.

Sorted sets in Redis are similar to the basic set data structure, except every element in
the set is associated with a floating point value, called the score. Like Redis sets,
sorted sets consist of a collection of unique elements but the set is ordered according to
two rules:

* Elements with different scores are ordered by score
* Elements with same scores are ordered by string comparison on the element

Using these two rules, the set is maintained in ascending order with the smallest element
at position zero. As of the 3.2 release, Redis provides 21 different commands to operate
on sorted sets. These commands allow the user to perform a variety of operations:

* Add and remove elements from the set
* Retrieve elements of the set based on score or element name
* Update element scores
* Combine multiple sets
* Determine the size of sets

You may also hear Redis sorted sets referred to as zsets.

## Blog Post Recommendations

In this chapter, we are going to look at building a system for quickly serving blog post
recommendations to users of our website. In our system, recommendations will be generated
from a variety of data sources that are processed off-line to generate a list of
recommendations that will then be loaded into Redis for serving to visitors.

The recommendation engine will deliver a new set of recommendations for users all at once.
The recommendations data will be structured as a `user_id` followed by a list of
`(page_id, score)` tuples. A variable number of recommendations is generated for each user.

## Data Storage Conventions

Our recommended blog posts will be stored in the database using the sorted set datatype.
The recommended blog posts for a particular user will be stored under the key 
`user:{user_id}:recommendations`.  The recommendation data will consist of the `page_id`
of the blog post and a score between -1.0 to 1.0, with 1.0 being highly recommended and a
-1.0 being not recommended.

To improve the clarity of our examples, the Notebook environment includes several utility 
functions.  In this chapter, the primary utility functions we use are:

* `user_recommendation_key` - generates the appropriate key for the user recommendations
* `clear_user_recommendations` - removes the recommendations for the given user

## Storing Recommendations

The first step in building our example recommendation system is building a system to load
the data from recommendation backend into Redis. As our code processes data from the
backend recommendation system it needs to:

* Parse the recommendation data
* Compute a key from the data
* Remove the key if it exists
* Store the recommendations in Redis

The Redis ZADD command adds the specified member plus score to the sorted set, creating
the sorted set if it doesn't exits. In our sample system, an entire batch of
recommendations is loaded at once, so we can't just use the ZADD command, otherwise we
would never clear the old recommendations.

*Run our example by selecting the cell and pressing SHIFT + ENTER*

In [ ]:
import redis

# example connection parameters 
config = {
    "host": "redis",
    "port": 6379
}

r = redis.StrictRedis(**config)

sample_user_recommendations = [
    (3001, [ (201, -1), (202, -.8), (203, .4), (204, .9), (205, .8), (206, .7)]),
    (3002, [ (201, -1), (202,  .7), (204, .3) ]),
    (3003, [ (201, -1), (202, -.2), (203, .4) ]),
    (3004, [ (201, -1), (202, -.3), (203, .2) ]),
    (3005, [ (201, -1), (202,  .8), (203, .2), (204, .9) ]),
    (3006, [ (201, -1), (202,  .8), (203, .1), (204, .9) ]),
    (3007, [ (201, -1), (202,  .2), (203, .1) ]),
    (3008, [ (201, -1), (202,  .1), (203, .6) ]),
    (3009, [ (201, -1), (202,  .7), (203, .6), (204, .9) ]),
    (3010, [ (201, -1), (202,  .2) ])
]

In [ ]:

def user_recommendation_key(user_id):
    "Returns the appropriate key for user recommendations"

    return "user:" + str(user_id) + ":recommendations"

def clear_user_recommendations(r, user_id):
    "Clears the recommendations for the given user"

    key = user_recommendation_key(user_id)
    r.delete(key)

def get_user_recommendations(r, user_id):
    "Returns the recommendations for the specified user"

    key = user_recommendation_key(user_id)
    return r.zrange(key, 0, -1, withscores=True)

def update_user_recommendations(r, user_id, recommendations):
    "Update or create a set of recommendations for the user"
    print("in update_user_recommendations with user_id=" + str(user_id))
    key = user_recommendation_key(user_id)
    # print("key is " + key)
    cnt = 0
    for page_pair in recommendations:
        r.zadd(key,{page_pair[0]:page_pair[1]})
        cnt += 1
    
    return cnt

def parse_recommendations(r, recommendation_list):
    "Parses a list of recommendations and updates for a given user"
    print("in parse_recommendations")
    for user_id, recommendations in recommendation_list:
        update_user_recommendations(r, user_id, recommendations)

clear_user_recommendations(r, 3001)
parse_recommendations(r, sample_user_recommendations)

print ("Recommendations loaded for user 3001: {}".format(get_user_recommendations(r, 3001)))


When you execute the sample code you should see a message indicating that 4 recommendations
were loaded for user 3001.  We can use our redisinsight to see the state of the database after loading the recommendations.

The signature for zadd changed from python 2 to python 3.  To store a recommendation in Redis for page 201 with a score .7, execute the command:

In [ ]:
r.zadd('sample_recommendation', {201 : .7})


## Retrieving Recommendations

The next step in building our sample recommendation system, is to serve up recommendations
to visitors. We are going to look at two different ways of providing recommendations to
the users: top N and above a threshold.

### Top N Retrieval

The first example method we are going to look at returns an arbitrary number of the best
recommendations available for a user. To retrieve recommendations in this fashion, our
code needs to:

* Compute a key for the requested user
* Retrieve the top ranked items

This can be accomplished with a single Redis command: ZREVRANGE. The ZREVRANGE command
returns a range of a sorted set, treating the set as if it was ordered from highest to
lowest. So if we use ZREVRANGE to request the range 0 to N-1, we get our *top* N elements.

*Run the sample code below using SHIFT + ENTER to see how ZREVRANGE works.*

In [ ]:

def top_recommendations(r, user_id, N):
    "Returns the top N recommendations for the user"
    
    key = user_recommendation_key(user_id)
    return r.zrevrange(key, 0, N - 1)
    
print ("Top 2 recommendations for user 3001: ", top_recommendations(r, 3001, 2))


After the code runs, you should see a message displaying the top two recommendations for
user 3001. Like many other Redis commands, ZREVRANGE gracefully handles missing data
instead of throwing an error. If the requested key does not exist an empty result is
returned and if the range requested is larger than the number of members available, the
smaller range is returned.

In this code example, we only return the members of the sorted set in reverse sorted
order. If we wanted to get both the members and the scores, we could add the `WITHSCORES`
modifier to the command to return both. In the *redis-py* library, this is accomplished by
passing the `withscores=True` argument to the function call.

* * *

> **Note**
> 
> Most of the examples in this exercise work with reverse ranges, because we are
> interested in the *top* recommendations for a user.  Redis provides "forward" versions 
> of all the reverse (REV) commands that we are using.  The forward versions treat the
> sorted set as ordered in the ascending direction.
>

* * *

### Threshold Recommendations 

The purpose of our recommendation system is to increase traffic to our website by pointing
out blog posts that will be particularly engaging for the user. Recommending the top 2, 5
or N recommendations works for many users, but it doesn't guarantee quality
recommendations for all our users.

Our recommendation system works by giving posts a score from -1 to 1, indiciating how
likely a user is to like the particular post, so there may be no strongly recommended
items for a given user. In that case, it might be better not to provide any
recommendations at all rather than a low scoring post. We can implement this version of
returning recommendations with code that:

* Compute a key for the requested user
* Return all keys above the requested threshold

This can also be accomplished with a single Redis command: ZREVRANGEBYSCORE. The
ZREVRANGEBYSCORE command is very similar to the ZREVRANGE command, it returns a list of
members in reverse order. The difference between the two commands is that ZREVRANGE takes
a range of elements while ZREVRANGEBYSCORE takes a range of scores to determine the
results.

In the next example, we return a number of recommendations but only return strong
recommendations that are above a provided threshold. *Run the example below by
selecting the code cell and pressing SHIFT + ENTER*

In [ ]:
def top_recommendations(r, user_id, threshold):
    "Returns the top recommendations for user based on a minimum score"
    
    key = user_recommendation_key(user_id)
    return r.zrevrangebyscore(key, 1.0, threshold, withscores=True)
    
print ("Top recommendations for user 3001: ", top_recommendations(r, 3001, .7))


You should see a list of the top recommendations (along with their scores) for user 3001.

Our threshold-based code returns all of the recommendations within a given range, but for
some users that could be a substantial number of recommendations. It would be nice to
continue to limit our results to a certain number of strongly recommended items. This can
be accomplished by adding the LIMIT argument to our command. The LIMIT argument takes an
offset and a count and applies it to our range.

The sample code below combines both the top-N and threshold approaches, to return the
top-N results above a certain threshold for a user. 

*Run the sample code by selecting the code cell and pressing SHIFT+ENTER*

In [ ]:
def top_recommendations(r, user_id, threshold, N=2):
    "Returns the top N (default=2) recommendations above the given threshold"
    
    key = user_recommendation_key(user_id)
    return r.zrevrangebyscore(key, 1.0, threshold, withscores=True, start=0, num=N)

print ("Top recommendations for user 3001: ", top_recommendations(r, 3001, .7))  

After executing the sample code, you should see two recommendations for user 3001 that are
both above the 0.7 score threshold.


* * *

> **Note**
>
> The scores passed to the ZREVRANGEBYSCORE command can be specified as intervals.  You 
> can use `(` to indicate an open or non-inclusive interval and `]` to indicate a closed
> or include interval.  Additionally, the arguments`+inf` and `-inf` can be used to
> indicate positive and negative infinity, which allows you to select all of the possible
> scores, without knowing the range beforehand.


* * *

## Retiring Recommendations

Once a user has visited a particular recommendation, it is much less valuable as a future
recommendation. Our recommendation system is supposed to increase the number of posts a
user reads, so recommending them the same posts over and over generally won't increase our
traffic. There are a different ways we could handle this - we could remove a
recommendation once a user visits it or we could reduce it's score make it less likely to
be served to the user.

### Removing Recommendations

The first way we can handle "used" recommendations is to remove the recommendation from the
sorted set. This can be accomplished with a single Redis command: ZREM. The ZREM command
removes a sequence of one or more members from a Sorted Set. The following example uses
ZREM to remove a recommendation once it is used:

In [ ]:
def use_recommendation(r, user_id, page_id):
    "Removes a recommendation for a user after they visit the recommendation"
    
    key = user_recommendation_key(user_id)
    r.zrem(key, page_id)
    

parse_recommendations(r, sample_user_recommendations)

print ("Top recommendations for user 3001: ", top_recommendations(r, 3001, .5, N=4))
use_recommendation(r, 3001, 206)
print ("Top recommendations for user 3001: ", top_recommendations(r, 3001, .5, N=4))


After running the above example, you should see the recommendations for user 3001 before removing page 208, then
after removing page 208.  

### Reducing Scores

Reducing the score of a recommendation once it's been visited is another way we can handle
a "used" recommendation. This way the strong recommendation is still in our set to
consider, but we give more weight to highly recommended items that haven't been viewed.
This can also be accomplished with a two commands: ZINCRBY and ZREMRANGEBYSCORE. The
ZINCRBY command takes a key, an increment, and a member as a parameter and updates the
member's score by the specified increment. The ZREMRANGEBYSCORE command is used to remove
the recommendation if it's score is less than -1. The ZREMRANGEBYSCORE command takes a
range of scores and removes all of the members in that range.

In our example below, we reduce the score of a post by .1 after it has been read, then
remove any post that has a score less than -1 to preserve our -1 to 1 range of scores:

In [ ]:

def use_recommendation(r, user_id, page_id):
    "Reduces the score of a recommendation after the specified user visits"
    
    key = user_recommendation_key(user_id)
    r.zincrby(key, -0.2, page_id)
    
    r.zremrangebyscore(key, "-inf", -1)

parse_recommendations(r, sample_user_recommendations)

print ("Top recommendations for user 3001: ", get_user_recommendations(r, 3001))
use_recommendation(r, 3001, 208)
print ("Top recommendations for user 3001: ", get_user_recommendations(r, 3001))


After running the above example, you should see the recommendations for user 3001 before
and after reducing the score for page 208.

## Working with Sets

In the examples we've looked at so far, nothing we've done has really distinguished Sorted
Sets from an ordered list, but Sorted Sets are called sets for a reason. Redis supports
set operations (operations on multiple sets) on Sorted Sets as well. Redis provides two
set operations on Sorted Sets:

* ZINTERSTORE - intersection and store
* ZUNIONSTORE - union and store

These operations perform the familiar set intersection or union operations and store the
results in a destination key. Arguments to the commands allow you to control how the
scores from the two sets are combined. Redis allows you to control how the scores for
members are combined. Scores can be combined using weights, aggregation (sum), or min/max.
See the [ZUNIONSTORE command](https://redis.io/commands/zunionstore) for detailed
information on combining scores.

# Review

This chapter looked at ways we can use Redis Sorted Sets to serve up recommendation data
to users. We first looked at loading recommendation data into Redis from an external
system, we then looked at two different ways of retrieving recommendations from Redis
(top-N and threshold), we finally looked at two ways (deletion and score reduction) to
retire used recommendation. Since most of our examples involved member manipulation, we
took a bit of time to look at the set operators that can be applied to Sorted Sets.

In the process, we learned how to use a variety of Redis commands:

* ZADD
* ZRANGE
* ZREVRANGE 
* ZRANGEBYSCORE
* ZREVRANGEBYSCORE
* ZINCRBY
* ZREMRANGEBYSCORE
* ZINTERSTORE
* ZUNIONSTORE

Redis provides a wide range of commands that modify Sorted Sets.  We've looked at
several of the commands in this chapter, but the details of all the commands available
can be found at [Sorted Set Documentation](https://redis.io/commands#sorted_set) on 
[Redis.io](https://www.redis.io).